## Goals

So, let us continue the sprucing and finally prepare the dataset for modeling (initially without concern for the other variables).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import regex as re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv('../data/cleaned_02_i.csv')
print(df.shape)
df.head()

(39565, 56)


,platform,text,hate_speech_score,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,...,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability
0,0,White people are trash.,0.46,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,"You've caught the big gay, it's known to be ai...",0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,"I'm not saying you said that, I'm just saying ...",-1.29,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Donald Trump. Yeet myself off a building onto ...,-0.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,Fabrice Fabrice is ostensibly black or black/l...,-2.84,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
i = -1

In [19]:
i+=1
df[df['hate_speech_score']<.5].iloc[i]['text']

"Won't happen. Birth rates are down.  Women want full control of reproduction. Feminists have very powerful lobbying.  There's no money in it for big pharma.  Men would be far more diligent and disciplined in taking it.  Too many reasons. The capability is already there."

https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech <br>
hate_speech_score - continuous hate speech measure, where higher = more hateful and lower = less hateful. > 0.5 is approximately hate speech, < -1 is counter or supportive speech, and -1 to +0.5 is neutral or ambiguous.

In [3]:
#First up is to rename the hate_speech_score to a more generic label.
df['hate_speech_score'].describe() #Again, recall the words copied above fom their website. So, for my purposes I'd be interested
#in potentially intervening in people's behaviour. Likely that would start the second category at least at .5;
#and we'd still need to figure out the gauge for the IMMEDIATE category.
#Part of what might be difficult is the subjectivity in joking and the like.More and more I'm thinking to perhaps go back and
#grab those other columns I ignored, such as 'genocide'..... However, for now when we're just doing language we'll work as is
#(anyways, i this case I'd rewrite/append to the quoted current source).



count    39565.000000
mean        -0.943911
std          2.073087
min         -8.340000
25%         -2.330000
50%         -0.700000
75%          0.570000
max          6.300000
Name: hate_speech_score, dtype: float64

In [6]:
df['hate_speech_score'].quantile(.95)

2.04

In [4]:
i = -1

In [5]:
i += 1
print(df.iloc[i, 2], i)
df.iloc[i, 1]

0.46 0


'White people are trash.'

### Introducing the first hyperparameter - 'benign', 'concerning', 'malevonent'
Corresponding they'll be known as 0, 1, and 2 respectively. However, please note - likely I may tweak these values later. Recall that initialy these 'predictions' are categorical and I'm somewhat artificially assigning such classes to them for my purposes.
A further difficulty will be the trend I'v already noticed of 'counter/supportive speach' often being quite graphc as well. Besides fom a NLP point of view making things more difficult, definitionally it would seem to me crass and crude to refer to things in such a boorish manner. Ie perhaps I am biased, however I would hope supportive speach is not as filthy as hate speach, with the only difference being the direction it is aimed at.

On the topic of other potential parameters: Pay attention to how I'll also, later on in this section, clean the various texts. Perhaps it'll be worthwhile to experiment with that.

Oh wait, let me copy over the fllowing:
hate_speech_score - continuous hate speech measure, where higher = more hateful and lower = less hateful. 
> 0.5 is approximately hate speech, < -1 is counter or supportive speech, and -1 to +0.5 is neutral or ambiguous.

...Misread this - ambiguous also goes below...


#...Misread this - ambiguous also goes below...

In [6]:
lower_limit, upper_limit = .5, 2
df['hate_rank'] = [2 if i > upper_limit
                  else 1 if i > lower_limit else 0
                  for i in df['hate_speech_score']]
df['hate_rank'].value_counts(normalize=True)

hate_rank
0    0.738304
1    0.208973
2    0.052723
Name: proportion, dtype: float64

In [7]:
df['hate_rank'].value_counts(normalize=True) #Hmm, eh whatever - we'll keep it at this propotion for now.
#To write it out in a comment here,in case I change it later: .44,.5, .06 resp.

hate_rank
0    0.738304
1    0.208973
2    0.052723
Name: proportion, dtype: float64

In [8]:
#Hmm, something else I noticed while going through posts is the useage of capitals. Perhaps we should go ahead and make a
#contains lots of capitals column:
samp = "OKAY, I'LL LEAVE YOU FUCK YOURSELF LIKE A FUCKING DOG BARKING, BECAUSE I DON'T GIVE A FUCK TO A FUCKING BITCH LIKE YOU! I HOPE YOUR DICK-FACE FALLS ON A PIECE OF CRAP, HAVE A CRAPPY DAY, JACKARSE"

In [9]:
def capital_checker(sentance, thresh=.2):
    count, per_count = 0, 0
    for i in range(len(sentance)):
        if sentance[i].isupper() == True:
            count += 1
        if sentance[i] == '.':
            per_count += 1
    if (count - per_count)/len(sentance) >= thresh and count > per_count: #Ie the sentance has an overwhelming capital presence...
        verdict = 1
    else:
        verdict = 0
    return verdict

In [10]:
df['capital_sentance'] = df['text'].apply(capital_checker)

In [11]:
df['capital_sentance'].value_counts(normalize=True)

capital_sentance
0    0.966056
1    0.033944
Name: proportion, dtype: float64

In [12]:
#Great, now we can move foreward with NLPing, finally.Pragmatically it might be worthwhile to playaround with not lowercasing
#the words, but for now we'll go with the 'default' approach of doing so, in an attempt to reduce the upcoming flood of featues.

#While I'm doing all this, I might as well make a word counter as well. That might be useful later.

#Hmm, let's start by breaking apart the comments, trying to get an aveage word count...
def document_words(raw_document):

    #Remove some various html hiccups that may have crept in.
    document_text = BeautifulSoup(raw_document).get_text()
    
    #Further purge it via regex to keep only characters per Regex. For sure in this case I'm keeping numbers
    letters_only = re.sub("[^a-zA-Z0-9]", " ", document_text)
    #Convert to lower case, split into individual words.
    
    words = letters_only.split()
    words = [w.lower() for w in words]
 
    p_stemmer = PorterStemmer()
    stemmed_words = [p_stemmer.stem(i, to_lowercase=False) for i in words]
    
    #And finally, let us merge back everything to a new document.
    return(" ".join(stemmed_words))

In [13]:
 df['cleaned_text'] = df['text'].apply(document_words)

C:\Users\bwolf\AppData\Local\Temp\ipykernel_10784\3250074403.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  document_text = BeautifulSoup(raw_document).get_text()


In [14]:
df['wc_cleaned_text'] =  df['cleaned_text'].str.split().str.len()

In [15]:
df['wc_cleaned_text'].describe()

count    39565.000000
mean        25.779932
std         20.312349
min          1.000000
25%         11.000000
50%         20.000000
75%         35.000000
max        128.000000
Name: wc_cleaned_text, dtype: float64

In [16]:
df.groupby('hate_rank')['wc_cleaned_text'].describe()#Hmm, pehaps there is a trend here afterall - 0 is longer on
#all fronts. Ie perhaps when one uses words more, as opposed to being rash and simply labeling things in a smaller word insult
#that can be easily digested, that is a sign of a more nuanced person. However,such differences are negligible so perhaps
#it is not all that significant. Regardless, we continue.

,count,mean,std,min,25%,50%,75%,max
hate_rank,,,,,,,,
0,29211.0,26.864948,20.683143,1.0,12.0,21.0,37.00,128.0
1,8268.0,22.280963,18.767788,2.0,9.0,16.0,30.00,124.0
2,2086.0,24.454458,19.301883,2.0,10.0,19.0,32.75,117.0


## A bit more EDA

Originally I ended here, but I want to concretize observations on different groups:

Note, to try to keep things a bit simpler, I'll focus on each target individually. Ie pick on just Jews as opposed to old disabled Jewish grandmas.

In [57]:
targets = list(df.columns[df.columns.str.contains("target")])
print(len(targets))
targets[:5]

53


['target_race_asian',
 'target_race_black',
 'target_race_latinx',
 'target_race_middle_eastern',
 'target_race_native_american']

In [61]:
target_df = []
for i in targets:
    print(f"{i}:")
    print(df.loc[(df['hate_rank']!=0) & (df[i]>0),'hate_speech_score'].describe())
    target_df.append(df.loc[(df['hate_rank']!=0) & (df[i]>0),'hate_speech_score'].describe())
    print('~'*45)

target_race_asian:
count    467.000000
mean       1.554904
std        0.797896
min        0.510000
25%        0.950000
50%        1.400000
75%        1.975000
max        5.140000
Name: hate_speech_score, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_race_black:
count    1635.000000
mean        1.475651
std         0.775794
min         0.510000
25%         0.890000
50%         1.310000
75%         1.890000
max         5.530000
Name: hate_speech_score, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_race_latinx:
count    707.000000
mean       1.513211
std        0.816515
min        0.510000
25%        0.905000
50%        1.310000
75%        1.955000
max        5.140000
Name: hate_speech_score, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_race_middle_eastern:
count    1180.000000
mean        1.556924
std         0.848909
min         0.510000
25%         0.910000
50%         1.360000
75%         1.990000
max         6.300000


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_disability:
count    649.000000
mean       1.500293
std        0.753337
min        0.510000
25%        0.900000
50%        1.400000
75%        1.930000
max        5.130000
Name: hate_speech_score, dtype: float64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [62]:
target_df

[count    467.000000
 mean       1.554904
 std        0.797896
 min        0.510000
 25%        0.950000
 50%        1.400000
 75%        1.975000
 max        5.140000
 Name: hate_speech_score, dtype: float64,
 count    1635.000000
 mean        1.475651
 std         0.775794
 min         0.510000
 25%         0.890000
 50%         1.310000
 75%         1.890000
 max         5.530000
 Name: hate_speech_score, dtype: float64,
 count    707.000000
 mean       1.513211
 std        0.816515
 min        0.510000
 25%        0.905000
 50%        1.310000
 75%        1.955000
 max        5.140000
 Name: hate_speech_score, dtype: float64,
 count    1180.000000
 mean        1.556924
 std         0.848909
 min         0.510000
 25%         0.910000
 50%         1.360000
 75%         1.990000
 max         6.300000
 Name: hate_speech_score, dtype: float64,
 count    257.000000
 mean       1.661245
 std        0.860369
 min        0.510000
 25%        1.030000
 50%        1.470000
 75%        2.1400

In [67]:
#target_df = pd.DataFrame(targets)
try_df = pd.DataFrame(target_df)
try_df['target'] = targets
try_df.set_index('target', inplace=True)

In [81]:
cats = ['target_race','target_religion','target_origin','target_gender','target_sexuality','target_age','target_disability']

In [96]:
try_df.loc[cats] #From the category pint of viw, approx the same. Notice that I did include anything in the '1' category
#so perhaps the extremes are differnet. We'll start iwth this though.

#Anyways, a bit the same. To keep things short I'll focus on means: Approx. the same with gender actually being the lowest.

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_race,3836.0,1.480792,0.795623,0.51,0.88,1.30,1.8900,6.30
target_religion,1637.0,1.491625,0.815088,0.51,0.88,1.29,1.8900,6.30
target_origin,2370.0,1.478342,0.794170,0.51,0.87,1.31,1.8875,6.30
target_gender,4871.0,1.425024,0.740792,0.51,0.85,1.28,1.8100,6.05
target_sexuality,2579.0,1.482133,0.746033,0.51,0.91,1.35,1.9000,6.09
target_age,357.0,1.461765,0.789327,0.51,0.83,1.31,1.9100,4.91
target_disability,649.0,1.500293,0.753337,0.51,0.90,1.40,1.9300,5.13


In [100]:
try_df[try_df['mean']==try_df['mean'].max()] #Oh, this is quite a shocker that the NAs really getting pounded.... 
#Note however the lowish count overall

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_race_native_american,257.0,1.661245,0.860369,0.51,1.03,1.47,2.14,5.14


In [101]:
try_df[try_df['mean']==try_df['mean'].min()] #Not surprising the overall low hate, given the assumption that most of the people
#posting were probably Christian, even if all of these are from people of other religion, that the overall hate would
#probably be less as the Christians are likely a significant majority of the population.

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_religion_christian,442.0,1.336244,0.702379,0.51,0.81,1.14,1.68,4.68


In [97]:
try_df[try_df.index.str.contains('religion')] #Oh wow, in this sample Jews actually aren't the most hated on... Unsurprised
#about the Chrstians (see earlier). However, those Eastern religions are up there... Now, they aren't the highest counts, but stll...
#Of note as well is how Muslims are ovr half the targets. Also of note is considering the global populatio of Jews how
#much more they get targeted proportionally...
    #Oh, notice the explicit overlap... ie add up the Abrahamic religions and you're already over the target's count...
        #Hence, these posts have overlaps. Regardless...

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_religion_atheist,132.0,1.430758,0.720145,0.51,0.8300,1.230,1.7825,3.91
target_religion_buddhist,130.0,1.514615,0.752224,0.51,0.9025,1.345,1.9300,3.91
target_religion_christian,442.0,1.336244,0.702379,0.51,0.8100,1.140,1.6800,4.68
target_religion_hindu,189.0,1.570317,0.788465,0.51,0.9900,1.380,2.1400,4.33
target_religion_jewish,471.0,1.532442,0.834226,0.51,0.8700,1.380,1.9300,5.09
target_religion_mormon,134.0,1.435672,0.709711,0.51,0.8225,1.270,1.7975,3.91
target_religion_muslim,897.0,1.522564,0.815932,0.51,0.9000,1.330,1.9400,6.30
target_religion_other,300.0,1.492433,0.851464,0.51,0.8275,1.265,1.9325,5.42
target_religion,1637.0,1.491625,0.815088,0.51,0.8800,1.290,1.8900,6.30


In [115]:
#I'll redo above just focusing on 2s:
try_df2 = []
for i in targets:
    #print(f"{i}:")
    #print(df.loc[(df['hate_rank']!=0) & (df[i]>0),'hate_speech_score'].describe())
    try_df2.append(df.loc[(df['hate_rank']==2) & (df[i]>0),'hate_speech_score'].describe())
    #print('~'*45)

In [116]:
try_df2 = pd.DataFrame(try_df2)
try_df2['target'] = targets
try_df2.set_index('target', inplace=True)

In [117]:
try_df2.loc[cats] #Unsurprising to me, via domain knowledge, that religion is the highest, followed by race and origin.
#Also, note the very low count for age...

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_race,802.0,2.705212,0.687918,2.01,2.22,2.47,2.950,6.30
target_religion,340.0,2.763824,0.699808,2.01,2.26,2.53,3.115,6.30
target_origin,488.0,2.711086,0.681568,2.01,2.21,2.50,2.980,6.30
target_gender,929.0,2.617513,0.625841,2.01,2.18,2.42,2.840,6.05
target_sexuality,544.0,2.605478,0.611257,2.01,2.18,2.38,2.840,6.09
target_age,80.0,2.638125,0.605603,2.03,2.22,2.42,2.860,4.91
target_disability,137.0,2.634307,0.617099,2.01,2.19,2.38,2.910,5.13


In [118]:
try_df2[try_df2.index.str.contains('religion')] #Once again, on a count basis the Jews and the Muslims...
#And, unsurprisingly, the Jews, now focusing on the 'extreme' category are the highest. Muslims, howevr, are stll quite high
#and so too 'other'. Of interest also is the now 'higher' Christian rank (compared to before). Of special  interst is
#the quite low atheiest score, which I suppose I could imagine as pragmatically they don't have anything to believe in
#to poke holes in, unlike any other religion definitionally. Plus, notic their fewer count. Mormons also are quite low,
#surprisingly.

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_religion_atheist,30.0,2.536667,0.443126,2.01,2.2700,2.380,2.6850,3.91
target_religion_buddhist,32.0,2.616250,0.412872,2.02,2.3475,2.540,2.7400,3.91
target_religion_christian,69.0,2.627246,0.575259,2.01,2.2000,2.480,2.7700,4.68
target_religion_hindu,51.0,2.657059,0.497057,2.02,2.3250,2.510,2.8450,4.33
target_religion_jewish,104.0,2.799519,0.655802,2.01,2.3150,2.605,3.2600,5.09
target_religion_mormon,28.0,2.562857,0.394094,2.02,2.3375,2.525,2.6750,3.91
target_religion_muslim,207.0,2.714396,0.678718,2.01,2.2450,2.490,2.9950,6.30
target_religion_other,70.0,2.758857,0.699701,2.01,2.2800,2.635,2.9375,5.42
target_religion,340.0,2.763824,0.699808,2.01,2.2600,2.530,3.1150,6.30


In [119]:
try_df2[try_df2.index.str.contains('race')] #A more popular topic than religion, likely because it's more tangible.
#And yeah, much more evenlydistributed here, which is interesting considering how many more counts there are ; eh
#perhaps that's already enough for the 'law of large numbers' to start spreading the love/hate evenly.

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_race_asian,112.0,2.699286,0.640468,2.01,2.2475,2.540,3.0100,5.14
target_race_black,338.0,2.668639,0.660095,2.01,2.2225,2.450,2.9000,5.53
target_race_latinx,161.0,2.723727,0.669972,2.01,2.2300,2.540,2.9600,5.14
target_race_middle_eastern,289.0,2.747232,0.728771,2.01,2.2500,2.490,3.0400,6.30
target_race_native_american,75.0,2.727333,0.709907,2.01,2.2400,2.540,2.9550,5.14
target_race_pacific_islander,58.0,2.701379,0.667815,2.01,2.2550,2.495,2.9075,5.14
target_race_white,170.0,2.657706,0.665298,2.01,2.2000,2.430,2.8100,5.09
target_race_other,159.0,2.735220,0.668774,2.01,2.2750,2.530,2.9500,5.57
target_race,802.0,2.705212,0.687918,2.01,2.2200,2.470,2.9500,6.30


In [122]:
try_df2[try_df2.index.str.contains('origin')] #I don't appreciate this cohort enough to comment.

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_origin_immigrant,203.0,2.689507,0.671297,2.01,2.1850,2.48,2.985,6.30
target_origin_migrant_worker,89.0,2.840449,0.739003,2.01,2.3100,2.70,3.100,6.30
target_origin_specific_country,338.0,2.683846,0.630364,2.01,2.2125,2.50,2.945,5.57
target_origin_undocumented,142.0,2.747535,0.706744,2.01,2.2100,2.55,3.055,6.30
target_origin_other,103.0,2.772233,0.684340,2.01,2.2700,2.58,3.005,5.57
target_origin,488.0,2.711086,0.681568,2.01,2.2100,2.50,2.980,6.30


In [123]:
try_df2[try_df2.index.str.contains('gender')] #Perhaps the term 'cis' didn't pickup. Regardless:
#Yeah.... woman are the clear winners/losers here. Once again, the hate is pretty evenly spread out.

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_gender_men,248.0,2.595605,0.607983,2.01,2.1600,2.395,2.8250,6.01
target_gender_non_binary,50.0,2.595400,0.575323,2.01,2.2025,2.395,2.7725,4.91
target_gender_transgender_men,89.0,2.621124,0.525829,2.01,2.2400,2.420,2.9200,4.87
target_gender_transgender_unspecified,64.0,2.634062,0.582201,2.01,2.2075,2.405,2.9200,4.87
target_gender_transgender_women,67.0,2.573433,0.558425,2.01,2.1500,2.390,2.7550,4.91
target_gender_women,731.0,2.615841,0.627028,2.01,2.1900,2.420,2.8300,6.05
target_gender_other,39.0,2.620513,0.561398,2.02,2.1450,2.440,2.8200,4.01
target_gender,929.0,2.617513,0.625841,2.01,2.1800,2.420,2.8400,6.05


In [124]:
try_df2[try_df2.index.str.contains('sexuality')]#Unsurprisigly the majority class, straight, is the lowest and 'other' is the highest,
#being the most foreign. Ie given the majority are attracted to the opposite it can be understod the relative
#lower hate as even their interests aren't inherently abnormal - just atypical for, say a man to lik a man - yet approximately
#half of the world likes men!
#Of interest though is th much highr lesbian hate than gay. Perhaps since they're also women (see above).

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_sexuality_bisexual,182.0,2.611923,0.551055,2.01,2.1725,2.440,2.9200,4.87
target_sexuality_gay,421.0,2.594371,0.597366,2.01,2.1800,2.380,2.8400,6.09
target_sexuality_lesbian,174.0,2.662069,0.618592,2.01,2.2125,2.480,2.9675,6.09
target_sexuality_straight,152.0,2.543421,0.538536,2.01,2.1600,2.335,2.8100,4.87
target_sexuality_other,66.0,2.763788,0.760380,2.02,2.1800,2.475,3.1250,5.78
target_sexuality,544.0,2.605478,0.611257,2.01,2.1800,2.380,2.8400,6.09


In [125]:
try_df2[try_df2.index.str.contains('age')] #Once again other.... and I'm also unsurprised that senios are lower as
#people are more willing to forgive somebody 'stuck in their ways' which the elderly classically and sterotypically are.

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_age_children,32.0,2.638750,0.583392,2.03,2.2175,2.46,2.7475,4.22
target_age_teenagers,41.0,2.601463,0.661939,2.03,2.1400,2.36,2.7300,4.91
target_age_young_adults,47.0,2.629362,0.546158,2.03,2.2300,2.42,2.9650,4.26
target_age_middle_aged,30.0,2.604667,0.495620,2.06,2.2625,2.42,2.7975,3.91
target_age_seniors,24.0,2.577917,0.452942,2.06,2.2550,2.42,2.7225,3.58
target_age_other,6.0,2.831667,0.663187,2.10,2.3475,2.83,3.0575,3.91
target_age,80.0,2.638125,0.605603,2.03,2.2200,2.42,2.8600,4.91


In [126]:
try_df2[try_df2.index.str.contains('disability')] #Particuarly puzzled here, but without going into the texts not to much to comment.
#However, cognitive is the clear dominant target. 

,count,mean,std,min,25%,50%,75%,max
target,,,,,,,,
target_disability_physical,15.0,2.581333,0.404137,2.10,2.2600,2.480,2.8050,3.38
target_disability_cognitive,92.0,2.617609,0.635151,2.01,2.1875,2.360,2.8800,5.13
target_disability_neurological,30.0,2.641000,0.549522,2.02,2.2425,2.430,2.9175,3.98
target_disability_visually_impaired,9.0,2.525556,0.343297,2.10,2.2700,2.420,2.6900,3.16
target_disability_hearing_impaired,9.0,2.525556,0.343297,2.10,2.2700,2.420,2.6900,3.16
target_disability_unspecific,44.0,2.557273,0.514521,2.01,2.2475,2.385,2.6750,4.40
target_disability_other,20.0,2.566500,0.476238,2.02,2.1850,2.365,2.9350,3.42
target_disability,137.0,2.634307,0.617099,2.01,2.1900,2.380,2.9100,5.13


In [120]:
cats

['target_race',
 'target_religion',
 'target_origin',
 'target_gender',
 'target_sexuality',
 'target_age',
 'target_disability']

In [17]:
#Hmm, yeah, I guess we'll export these
df.to_csv('../data/cleaned_02_ii.csv', index=False)